In [1]:
from datasets import load_dataset

dataset = load_dataset('versae/bibles', 'testament')
dataset

Found cached dataset bibles (/home/kiddos/huggingface/datasets/versae___bibles/testament/1.0.0/896439b9ad0fc0fb6df27a9146d8823e457a75f06e1a90169728cb690a531248)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'language', 'year', 'century', 'codebook'],
        num_rows: 1570633
    })
    validation: Dataset({
        features: ['text', 'label', 'language', 'year', 'century', 'codebook'],
        num_rows: 216493
    })
    test: Dataset({
        features: ['text', 'label', 'language', 'year', 'century', 'codebook'],
        num_rows: 216511
    })
})

In [2]:
dataset = dataset.filter(lambda x: x["language"] == 'CHI')
dataset

Loading cached processed dataset at /home/kiddos/huggingface/datasets/versae___bibles/testament/1.0.0/896439b9ad0fc0fb6df27a9146d8823e457a75f06e1a90169728cb690a531248/cache-e0bbf6e18a061e00.arrow
Loading cached processed dataset at /home/kiddos/huggingface/datasets/versae___bibles/testament/1.0.0/896439b9ad0fc0fb6df27a9146d8823e457a75f06e1a90169728cb690a531248/cache-c4a4fdb95491d310.arrow
Loading cached processed dataset at /home/kiddos/huggingface/datasets/versae___bibles/testament/1.0.0/896439b9ad0fc0fb6df27a9146d8823e457a75f06e1a90169728cb690a531248/cache-039fcd18d6c43b3d.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'language', 'year', 'century', 'codebook'],
        num_rows: 26226
    })
    validation: Dataset({
        features: ['text', 'label', 'language', 'year', 'century', 'codebook'],
        num_rows: 3618
    })
    test: Dataset({
        features: ['text', 'label', 'language', 'year', 'century', 'codebook'],
        num_rows: 3619
    })
})

In [3]:
dataset['train'][0]

{'text': '但 现 在 你 们 要 弃 绝 这 一 切 的 事 ， 以 及 恼 恨 、 忿 怒 、 恶 毒 （ 或 作 ： 阴 毒 ） 、 毁 谤 ， 并 口 中 污 秽 的 言 语 。',
 'label': 0,
 'language': 'CHI',
 'year': 0,
 'century': 'unk',
 'codebook': 'COL'}

In [4]:
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer

model_name = 'bert-base-chinese'
tokenizer = AutoTokenizer.from_pretrained(model_name)

2023-01-29 20:08:27.519134: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def tokenize_function(example):
    return tokenizer(example['text'].replace(' ', ''))


# Use batched=True to activate fast multithreading!
tokenized_datasets = dataset.map(
    tokenize_function, remove_columns=['text', 'label', 'language', 'year', 'century', 'codebook']
)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 26226
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3618
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3619
    })
})

In [6]:
chunk_size = 256

def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // chunk_size) * chunk_size
    result = {}
    for k, t in concatenated_examples.items():
        result[k] = [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result


lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets = lm_datasets.remove_columns(['token_type_ids'])
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3825
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 523
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 533
    })
})

In [7]:
tokenizer.decode(lm_datasets['train'][0]['input_ids'])

'[CLS] 但 现 在 你 们 要 弃 绝 这 一 切 的 事 ， 以 及 恼 恨 、 忿 怒 、 恶 毒 （ 或 作 ： 阴 毒 ） 、 毁 谤 ， 并 口 中 污 秽 的 言 语 。 [SEP] [CLS] 說 話 的 時 候 、 有 一 個 法 利 賽 人 請 耶 穌 同 他 喫 飯 ． 耶 穌 就 進 去 坐 席 。 [SEP] [CLS] 虽 然 是 这 样 ， 这 些 醉 生 梦 死 的 人 还 是 照 样 玷 污 身 体 ， 藐 视 主 权 ， 毁 谤 尊 荣 。 [SEP] [CLS] 在 他 纳 和 米 吉 多 ， 并 靠 近 撒 拉 他 拿 、 耶 斯 列 下 边 的 伯 善 全 地 ， 从 伯 善 到 亚 伯 米 何 拉 直 到 约 念 之 外 ， 冇 亚 希 律 的 儿 子 巴 拿 ； [SEP] [CLS] 以 後 再 沒 有 咒 詛 ． 在 城 裡 有 神 和 羔 羊 的 寶 座 ． 他 的 僕 人 都 要 事 奉 他 ． [SEP] [CLS] 因 為 他 沒 有 甚 麼 償 還 之 物 、 主 人 吩 咐 把 他 和 他 妻 子 兒 女 、 並 一 切 所 有 的 都 賣 了 償 還 。 [SEP] [CLS] 约 坦 登 基 的 时 候 ， 是 二 十 五 岁 ； 他 在 耶'

In [8]:
tokenizer.decode(lm_datasets['train'][0]['labels'])

'[CLS] 但 现 在 你 们 要 弃 绝 这 一 切 的 事 ， 以 及 恼 恨 、 忿 怒 、 恶 毒 （ 或 作 ： 阴 毒 ） 、 毁 谤 ， 并 口 中 污 秽 的 言 语 。 [SEP] [CLS] 說 話 的 時 候 、 有 一 個 法 利 賽 人 請 耶 穌 同 他 喫 飯 ． 耶 穌 就 進 去 坐 席 。 [SEP] [CLS] 虽 然 是 这 样 ， 这 些 醉 生 梦 死 的 人 还 是 照 样 玷 污 身 体 ， 藐 视 主 权 ， 毁 谤 尊 荣 。 [SEP] [CLS] 在 他 纳 和 米 吉 多 ， 并 靠 近 撒 拉 他 拿 、 耶 斯 列 下 边 的 伯 善 全 地 ， 从 伯 善 到 亚 伯 米 何 拉 直 到 约 念 之 外 ， 冇 亚 希 律 的 儿 子 巴 拿 ； [SEP] [CLS] 以 後 再 沒 有 咒 詛 ． 在 城 裡 有 神 和 羔 羊 的 寶 座 ． 他 的 僕 人 都 要 事 奉 他 ． [SEP] [CLS] 因 為 他 沒 有 甚 麼 償 還 之 物 、 主 人 吩 咐 把 他 和 他 妻 子 兒 女 、 並 一 切 所 有 的 都 賣 了 償 還 。 [SEP] [CLS] 约 坦 登 基 的 时 候 ， 是 二 十 五 岁 ； 他 在 耶'

In [9]:
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling


batch_size = 8
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
train_dataloader = DataLoader(
    lm_datasets['train'],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

eval_dataloader = DataLoader(
    lm_datasets['validation'],
    batch_size=batch_size,
    collate_fn=data_collator,
)

for example_batch in train_dataloader:
    break

print(example_batch['input_ids'].shape)
print(example_batch['attention_mask'].shape)
print(example_batch['labels'].shape)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([8, 256])
torch.Size([8, 256])
torch.Size([8, 256])


In [10]:
print(example_batch['input_ids'][0])
print(example_batch['labels'][0])
print(tokenizer.decode(example_batch['input_ids'][0]))
# print(tokenizer.decode(example_batch['labels'][0]))

tensor([ 3221,  6905,  6121,  2345,  2769,  3341,  5442,  4638,  3192,   103,
          103,   976,   103,   800,  4638,  2339,   511,   102,   101,  4310,
         1922,   782,  1348,  2897,  6629,  4767,  1928,   103,  2802,   103,
          511,   102,   101, 16449,  3341,  6963,  3221,   855,  1146,  1920,
         4638,  5314,   855,  1146,  2207,  4638,  4867,  4886,  8024,  6821,
          103,  3690,   103,  4542,  7309,  4638,   511,   102,   101,   679,
         6814,  8024,   872,   103,  6206,   103,  2127,  2347,  5307,  2533,
         4708,  4638,  8024,   103,  1168,   103,  3341,   511,   102,   101,
         5635,   754,  2626,  3471,  8024,   800,  4638,  2797,  3667,  3221,
          103,  2626,  4638,  8024,   800,  1745,  6450,  2626,   103,  8024,
         4500,  5994,   969,  4638,   103,   103,  3673,  4127,  1737,  5736,
         4638,   782,  8024,  1315,   886,  1762,   103,   726,   103,  6382,
         6596,  4415,  4638,  3198,   952,  8024,   800,   738, 

In [11]:
from accelerate import Accelerator
from torch.optim import AdamW

model = AutoModelForMaskedLM.from_pretrained(model_name)

learning_rate = 1.25e-5
accelerator = Accelerator()
optimizer = AdamW(model.parameters(), lr=learning_rate)
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
import torch

def run_evaluation():
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    try:
        perplexity = torch.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")
    return perplexity


run_evaluation()

tensor(7.6577, device='cuda:0')

In [13]:
from transformers import get_scheduler

epochs = 10
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [14]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
output_dir = f'{model_name}-finetuned-bible-mlm'

def train():
    for epoch in range(epochs):
        # Training
        model.train()
        for batch in train_dataloader:
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

        perplexity = run_evaluation()
        print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

        # Save and upload
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)


train()

  0%|          | 0/4790 [00:00<?, ?it/s]

>>> Epoch 0: Perplexity: 4.772298336029053
>>> Epoch 1: Perplexity: 4.474167823791504
>>> Epoch 2: Perplexity: 4.149725914001465
>>> Epoch 3: Perplexity: 4.04140567779541
>>> Epoch 4: Perplexity: 3.9288315773010254
>>> Epoch 5: Perplexity: 3.920513868331909
>>> Epoch 6: Perplexity: 3.9138827323913574
>>> Epoch 7: Perplexity: 3.7478549480438232
>>> Epoch 8: Perplexity: 3.6737732887268066
>>> Epoch 9: Perplexity: 3.689364194869995


In [15]:
from transformers import pipeline

model.to('cpu')
fill_masker = pipeline('fill-mask', model=model, tokenizer=tokenizer)

In [16]:
for test_batch in eval_dataloader:
    break

print(test_batch['input_ids'][-1])
print(tokenizer.decode(example_batch['input_ids'][-1]))

tensor([ 1923,  6432,  8038,  5023,   103,  2094,  3171,   749,  1959,  8024,
         2769,   912,  2372,   103,   677,   103,  3308,  6224,  5456,   103,
         1290,  8024,   886,   800,   103,   103,   857,  1762,  6929,  7027,
          511,   102,   101,  5456,  4947,  1726,  5031,  6303,   510,  2769,
          738,  6206,  1558,   103,   103,   671,  1368,  6282,  8026,   872,
          947,   684,  1440,  6260,  2769,  8026,   102,   101,   103,  1728,
         4881,   103,  8024,   103,   103,  6763,  2483,   103,  1213,   103,
         2769,  4638,  6716,   860,  3867,  4607,   103,  2397,   511,   102,
          101,  1728,  3634,  8024,  6832,  6206,  2650,   103,  8024,  1762,
          677,  4638,  1921,   738,  2553,  7946,  3266,  8039,  1728,   711,
         2769,   103,   103,  1139,  8024,  2769,  2692,  2347,  2137,  8024,
         2553,   679,  3204,  2637,  8024,   738,   679,  6760,  2692,   679,
          976,   511,   102,   101,  2769,   738,  4761,  6887, 

In [17]:
fill_masker('耶 [MASK] 回 答 ： [UNK] [MASK] 神 [MASK] 差 来 的')

[[{'score': 0.971808135509491,
   'token': 4933,
   'token_str': '稣',
   'sequence': '[CLS] 耶 稣 回 答 ： [UNK] [MASK] 神 [MASK] 差 来 的 [SEP]'},
  {'score': 0.008269641548395157,
   'token': 2787,
   'token_str': '户',
   'sequence': '[CLS] 耶 户 回 答 ： [UNK] [MASK] 神 [MASK] 差 来 的 [SEP]'},
  {'score': 0.004375677090138197,
   'token': 4947,
   'token_str': '穌',
   'sequence': '[CLS] 耶 穌 回 答 ： [UNK] [MASK] 神 [MASK] 差 来 的 [SEP]'},
  {'score': 0.00393082806840539,
   'token': 6662,
   'token_str': '路',
   'sequence': '[CLS] 耶 路 回 答 ： [UNK] [MASK] 神 [MASK] 差 来 的 [SEP]'},
  {'score': 0.0015573027776554227,
   'token': 1469,
   'token_str': '和',
   'sequence': '[CLS] 耶 和 回 答 ： [UNK] [MASK] 神 [MASK] 差 来 的 [SEP]'}],
 [{'score': 0.6156852841377258,
   'token': 794,
   'token_str': '从',
   'sequence': '[CLS] 耶 [MASK] 回 答 ： [UNK] 从 神 [MASK] 差 来 的 [SEP]'},
  {'score': 0.32717201113700867,
   'token': 3221,
   'token_str': '是',
   'sequence': '[CLS] 耶 [MASK] 回 答 ： [UNK] 是 神 [MASK] 差 来 的 [SEP]'},
  {'score': 

In [18]:
fill_masker('来 到 希 [MASK] 家 那 里 ， 将 拉 [MASK] [MASK] 基 的 话 [MASK] 诉 了 他 。')

[[{'score': 0.8843076825141907,
   'token': 6205,
   'token_str': '西',
   'sequence': '[CLS] 来 到 希 西 家 那 里 ， 将 拉 [MASK] [MASK] 基 的 话 [MASK] 诉 了 他 。 [SEP]'},
  {'score': 0.025877993553876877,
   'token': 5384,
   'token_str': '罗',
   'sequence': '[CLS] 来 到 希 罗 家 那 里 ， 将 拉 [MASK] [MASK] 基 的 话 [MASK] 诉 了 他 。 [SEP]'},
  {'score': 0.020009057596325874,
   'token': 1239,
   'token_str': '勒',
   'sequence': '[CLS] 来 到 希 勒 家 那 里 ， 将 拉 [MASK] [MASK] 基 的 话 [MASK] 诉 了 他 。 [SEP]'},
  {'score': 0.013175214640796185,
   'token': 2526,
   'token_str': '律',
   'sequence': '[CLS] 来 到 希 律 家 那 里 ， 将 拉 [MASK] [MASK] 基 的 话 [MASK] 诉 了 他 。 [SEP]'},
  {'score': 0.011612769216299057,
   'token': 1392,
   'token_str': '各',
   'sequence': '[CLS] 来 到 希 各 家 那 里 ， 将 拉 [MASK] [MASK] 基 的 话 [MASK] 诉 了 他 。 [SEP]'}],
 [{'score': 0.1964188665151596,
   'token': 3683,
   'token_str': '比',
   'sequence': '[CLS] 来 到 希 [MASK] 家 那 里 ， 将 拉 比 [MASK] 基 的 话 [MASK] 诉 了 他 。 [SEP]'},
  {'score': 0.09126697480678558,
   'token': 843,